In [0]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation, GlobalAveragePooling2D
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam,SGD
# from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
# from keras.regularizers import l2

# import numpy as np


import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, Activation,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.regularizers import l2

import numpy as np

import os


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
#from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True

# Create a session with the above options specified.
#k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 100
num_classes = 10
epochs = 250
l = 12
num_filter = 24
growth_rate = 12
compression = 0.5
dropout_rate = 0.0

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train/255.
x_test = x_test/255.

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.25):
    global compression
    temp = input
    for _ in range(l):
        #Bottleneck layer
        BatchNorm_BN = BatchNormalization()(temp)
        relu_BN      = Activation('relu')(BatchNorm_BN)
        Conv2D_BN    = Conv2D(int(num_filter*4), (1,1), use_bias=False ,padding='same',
                           kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(relu_BN)
        BatchNorm    = BatchNormalization()(Conv2D_BN)
        relu         = Activation('relu')(BatchNorm)
        Conv2D_3_3   = Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same',
                           kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.25):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter), (1,1), use_bias=False ,padding='same',
                               kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2), strides=(2, 2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization(axis=-1, epsilon=1.1e-5)(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = GlobalAveragePooling2D()(relu)
    #flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(AvgPooling)
    
    return output

In [0]:
l = 12
num_filter = 24
growth_rate = 12
dropout_rate = 0.0

input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same',
                      kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(input)

First_Block = add_denseblock(First_Conv2D, growth_rate, dropout_rate)
num_filter+=(l*growth_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, growth_rate, dropout_rate)
num_filter+=(l*growth_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

# Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
# Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Second_Transition,  growth_rate, dropout_rate)
output = output_layer(Last_Block)


In [9]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 24)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 24)   0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [0]:
img_aug = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=5./32,
            height_shift_range=5./32,
            zoom_range=0.2,
            horizontal_flip=True)

img_aug.fit(x_train, seed=0)

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=.1,momentum=.9,nesterov=True),
              metrics=['accuracy'])


In [0]:

model_checkpoint= ModelCheckpoint("weights_file.hdf5", monitor="val_acc", save_best_only=True,
                                  save_weights_only=True, verbose=1)

def step_decay(epoch):
	lrate = 0.1
	if epoch >=50  and epoch < 75:
		lrate = lrate / 10
	elif epoch >= 75:
		lrate = lrate / 100	
	return float(lrate)

lrschedular = LearningRateScheduler(step_decay)

callbacks=[lrschedular]



In [13]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(model,
                                              strategy = tf.contrib.tpu.TPUDistributionStrategy(
                                                  tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://'+os.environ['COLAB_TPU_ADDR'])
                                              )
                                             )

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.59.236.226:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 8329416748504613873)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11741954000072704397)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 11104408832346881790)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7983252741256141336)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10765561497592262840)
INFO:tensorflow:*** Available Device: _Devi

In [14]:
tpu_model.fit_generator(img_aug.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=len(x_train) // batch_size,
                    epochs=200,validation_data=(x_test, y_test), validation_steps=x_test.shape[0] // batch_size,callbacks=callbacks)

Epoch 1/200
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(12,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(12, 32, 32, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(12, 10), dtype=tf.float32, name='dense_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning SGD {'lr': 0.10000000149011612, 'momentum': 0.8999999761581421, 'decay': 0.0, 'nesterov': True}
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:KerasCrossShard: <tensorflow.python.keras.optimizers.SGD object at 0x7fe087056f28> []
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 76.9868848323822 secs
INFO:tensorflow:Setting weights on TPU model.
INFO:tensorflow:CPU -> TPU lr: 0.10000000149011612 {0.1}
INFO:tensorflow:CPU -> TPU momentum: 0.8999999761581421 {0.9}
INFO:tensorflow:CPU -> TPU decay: 0.0 {0.0}
499/500 [============================>.] - ETA: 0s - loss: 2.0581 - acc: 0.

In [15]:
# Test the model
score = tpu_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id_10'), TensorSpec(shape=(4, 32, 32, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(4, 10), dtype=tf.float32, name='dense_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:KerasCrossShard: <tensorflow.python.keras.optimizers.SGD object at 0x7fe0682b4438> []
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 17.36962604522705 secs
 9952/10000 [============================>.] - ETA: 0sINFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(2,), dtype=tf.int32, name='core_id_10'), TensorSpec(shape=(2, 32, 32, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(2, 10), dtype=tf.float32, name='dense_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder f

In [17]:
# Save the trained weights in to .h5 format
tpu_model.save_weights("DNST_model_9454.h5")
print("Saved model to disk")

INFO:tensorflow:Copying TPU weights to the CPU
INFO:tensorflow:TPU -> CPU lr: 0.0010000000474974513
INFO:tensorflow:TPU -> CPU momentum: 0.8999999761581421
INFO:tensorflow:TPU -> CPU decay: 0.0
INFO:tensorflow:TPU -> CPU nesterov: True
Saved model to disk


In [0]:
from google.colab import files
files.download("DNST_model_9341.h5")